In [6]:
import os

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
!ls ../input/

print("\nEmbeddings:")
!ls ../input/embeddings/

embeddings  sample_submission.csv  test.csv  train.csv

Embeddings:
GoogleNews-vectors-negative300	paragram_300_sl999
glove.840B.300d			wiki-news-300d-1M


### Embeddings

* GoogleNews-vectors-negative300 - https://code.google.com/archive/p/word2vec/
* glove.840B.300d - https://nlp.stanford.edu/projects/glove/
* paragram_300_sl999 - https://cogcomp.org/page/resource_view/106
* wiki-news-300d-1M - https://fasttext.cc/docs/en/english-vectors.html

In [8]:
print('File sizes')
for f in os.listdir('../input'):
    if 'zip' not in f:
        print(f.ljust(30) + str(round(os.path.getsize('../input/' + f) / 1000000, 2)) + 'MB')

File sizes
embeddings                    0.0MB
train.csv                     124.21MB
sample_submission.csv         1.3MB
test.csv                      5.24MB


## Open trainset and testset

In [9]:
train = pd.read_csv('../input/train.csv').fillna(' ')
test = pd.read_csv('../input/test.csv').fillna(' ')

In [10]:
print("Shape of training set: ", train.shape)
print("Shape of test set: ", test.shape)

train_target = train['target'].values
np.unique(train_target)
print("\nPercentage of insincere questions irt sincere questions: ", train_target.mean(), "%")

Shape of training set:  (1306122, 3)
Shape of test set:  (56370, 2)

Percentage of insincere questions irt sincere questions:  0.06187017751787352 %


In [11]:
train.sample(10)

,qid,question_text,target
1179384,e71cb414144a4b739c7e,What are some causes for clubfoot?,0
342040,43004a88ac93dc331565,I recently got expelled from my middle school....,0
120092,17831e37027065a79bc6,What's the difference between someone who is p...,0
662618,81c95ec0f2d759589f67,Whatis cutt of physics ph.d. programme for tifr?,0
802058,9d28bbc89ce720ad698e,What are all the types of fascia and where do ...,0
111343,15cad2f61bbc6e43efc0,Will indigenous tribes in the United States al...,0
348916,445ff6e09a2e15a84ddc,"What, exactly is ""Model United Nations""?",0
1132349,ddedab4a5a6300177f03,How do I make a glitter tornado in a bottle?,0
12559,02776e8e68b96b9cdc6a,What are the easy steps to overcome depression?,0
433152,54e2fe65a2899a40d655,Can we lose weight by eating fruit?,0


In [12]:
test.sample(10)

,qid,question_text
28997,82cdbefd57feb49c9311,What is the best way to promote a new startup?
22241,64ab640396581e78f91b,What function of PWD?
25588,73959d8c209ce95e40fd,What are the chances of finding a bride for me...
52586,eea951548d23e141f7f9,Why are streaks of light and sound produced wh...
24153,6d0c23d2514a33796dcc,Can a dependent of subclass 400 visa holder wo...
1602,077c2113d4da435784f1,What is social proof theory?
13583,3d73020e8525753e9c79,Do you know the product of Mona Lisa luxury 10...
13764,3e79c4e4d80b9e2d4805,Where can I get help with my bills?
39667,b3547463d0ce84d25cc0,What is the minimum requirement for Master stu...
51285,e87f15d7f1f3a131839f,What is the situation of job market in New Zea...


In [13]:
insincere_q = train[train["target"] == 1]["question_text"].tolist()

with open('insinceres.txt', 'w') as f:
    for item in insincere_q:
        f.write("%s\n" % item)

## N-gram analysis

In [14]:
from collections import defaultdict
from nltk.corpus import stopwords
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

stop_words = set(stopwords.words('english')) 
insinc_df = train[train.target==1]
sinc_df = train[train.target==0]

def plot_ngrams(n_grams):

    ## custom function for ngram generation ##
    def generate_ngrams(text, n_gram=1):
        token = [token for token in text.lower().split(" ") if token != "" if token not in stop_words]
        ngrams = zip(*[token[i:] for i in range(n_gram)])
        return [" ".join(ngram) for ngram in ngrams]

    ## custom function for horizontal bar chart ##
    def horizontal_bar_chart(df, color):
        trace = go.Bar(
            y=df["word"].values[::-1],
            x=df["wordcount"].values[::-1],
            showlegend=False,
            orientation = 'h',
            marker=dict(
                color=color,
            ),
        )
        return trace

    def get_bar(df, bar_color):
        freq_dict = defaultdict(int)
        for sent in df["question_text"]:
            for word in generate_ngrams(sent, n_grams):
                freq_dict[word] += 1
        fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
        fd_sorted.columns = ["word", "wordcount"]
        trace = horizontal_bar_chart(fd_sorted.head(10), bar_color)
        return trace    

    trace0 = get_bar(sinc_df, 'blue')
    trace1 = get_bar(insinc_df, 'blue')

    # Creating two subplots
    if n_grams == 1:
        wrd = "words"
    elif n_grams == 2:
        wrd = "bigrams"
    elif n_grams == 3:
        wrd = "trigrams"
    
    fig = tools.make_subplots(rows=1, cols=2, vertical_spacing=0.04,
                              subplot_titles=["Frequent " + wrd + " of sincere questions", 
                                              "Frequent " + wrd + " of insincere questions"])
    fig.append_trace(trace0, 1, 1)
    fig.append_trace(trace1, 1, 2)
    fig['layout'].update(height=500, width=1150, paper_bgcolor='rgb(233,233,233)', title=wrd + " Count Plots")
    py.iplot(fig, filename='word-plots')


In [15]:
plot_ngrams(1)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [16]:
plot_ngrams(2)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [17]:
plot_ngrams(3)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [ ]:
## Number of words in the text
train["num_words"] = train["question_text"].apply(lambda x: len(str(x).split()))
test["num_words"] = test["question_text"].apply(lambda x: len(str(x).split()))

## Number of unique words in the text
train["num_unique_words"] = train["question_text"].apply(lambda x: len(set(str(x).split())))
test["num_unique_words"] = test["question_text"].apply(lambda x: len(set(str(x).split())))

## Number of characters in the text
train["num_chars"] = train["question_text"].apply(lambda x: len(str(x)))
test["num_chars"] = test["question_text"].apply(lambda x: len(str(x)))

## Number of stopwords in the text
train["num_stopwords"] = train["question_text"].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))
test["num_stopwords"] = test["question_text"].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))

## Average length of the words in the text
train["mean_word_len"] = train["question_text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test["mean_word_len"] = test["question_text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
## Truncate some extreme values for better visuals ##
train['num_words'].loc[train['num_words']>50] = 50
train['num_unique_words'].loc[train['num_unique_words']>50] = 50
train['num_chars'].loc[train['num_chars']>300] = 300
train['mean_word_len'].loc[train['mean_word_len']>10] = 10

f, axes = plt.subplots(5, 1, figsize=(15,40))

sns.boxplot(x='target', y='num_words', data=train, ax=axes[0])
axes[0].set_xlabel('Target', fontsize=12)
axes[0].set_title("Number of words in each class", fontsize=15)

sns.boxplot(x='target', y='num_unique_words', data=train, ax=axes[1])
axes[1].set_xlabel('Target', fontsize=12)
axes[1].set_title("Number of unique words in each class", fontsize=15)

sns.boxplot(x='target', y='num_chars', data=train, ax=axes[2])
axes[2].set_xlabel('Target', fontsize=12)
axes[2].set_title("Number of characters in each class", fontsize=15)

sns.boxplot(x='target', y='num_stopwords', data=train, ax=axes[3])
axes[3].set_xlabel('Target', fontsize=12)
axes[3].set_title("Number of stopwords in each class", fontsize=15)

sns.boxplot(x='target', y='mean_word_len', data=train, ax=axes[4])
axes[4].set_xlabel('Target', fontsize=12)
axes[4].set_title("Mean word length in each class", fontsize=15)

plt.show()

In [ ]:
print(train.columns)
train.head()

### References

* [General EDA](https://www.kaggle.com/tunguz/just-some-simple-eda)
* [Exploration notebook](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-qiqc)